# Introduction to Transpilation 


In [14]:
# Required packages
# %pip install qiskit
# %pip install qiskit-ibm-runtime
# %pip install qiskit-ibm-transpiler
# %pip install qiskit-aer
# %pip install colorama
# %pip install git+https://github.com/qiskit-community/Quantum-Challenge-Grader.git

useful functions

In [16]:
from colorama import Fore, Style
from utils import score_func
import numpy as np
def circuit_characteristics(circuit):
    # Extract information from the transpiled circuit
    circuit = circuit.decompose(reps=3)

    # Output various metrics about the transpiled circuit
    cops = circuit.count_ops()
    print(f'Count Ops: {cops}')
    cd = circuit.depth()
    print(f'Total circuit depth: {cd}')
    c2qd = circuit.size(lambda x: x.operation.num_qubits == 2)
    print(f'Number of 2-qubit gates: {c2qd}')

    # Output the score and depth specifically for 2-qubit gates, in color
    print(Fore.MAGENTA + Style.BRIGHT + f'2-qubit gate depth: {circuit.depth(lambda x: x.operation.num_qubits == 2)}')
    print(Fore.GREEN + f'Score: {score_func(circuit)}')
    
    return cops, cd, c2qd


def save_pm_settings(qc):
    l0 = 'name, last_name,init_method,initial_layout,layout_method,routing_method,translation_method,scheduling_method,optimization_method,seed,circuit_depth,2q_depth'

    _, cd, c2qd = circuit_characteristics(qc)
    variables_list  = [your_name, your_last_name,  init_method, initial_layout, layout_method, routing_method, translation_method, scheduling_method,  optimization_method, seed, cd, c2qd]
    variables_list2 =[]
    for i in variables_list:
        # print(i)
        if type(i)== list:
            print(i)

            variables_list2.append('[' + "-".join(map(str, [1,2,3])) + '],')
        else:
            variables_list2.append('{},'.format(i))

    print(variables_list2)
    l1 = ''.join(variables_list2)
    with open('./my_pm_data.csv','w+') as outfile:
        outfile.write(l0)
        outfile.write('\n')
        outfile.write(l1)


        

## Call Service

In [9]:
from qiskit_ibm_runtime import QiskitRuntimeService
service = QiskitRuntimeService()
backend = service.backend("ibm_brisbane")

## 1. Definition of the Problem
### Graph definition

In [ ]:
from networkx import barabasi_albert_graph, draw
#######################################################
#######################################################
#######################################################
n_nodes = 50
graph   = barabasi_albert_graph(n    = n_nodes, 
                                m    = 7, 
                                seed = 42)

draw(graph, with_labels=True)

<!-- # Brief Overview: Introduction to the Quantum Approximate Optimization Algorithm (QAOA) -->

<!-- ## Quick Intro: What is QAOA?

The Quantum Approximate Optimization Algorithm (QAOA) is a quantum algorithm designed to tackle combinatorial optimization problems. These are problems where the goal is to find the optimal solution from a set of finite possible outcomes that are often encountered in tasks like routing, scheduling, or resource allocation. QAOA operates by applying quantum operations that approximate the optimal solution, making it a promising approach for potentially solving these problems efficiently on near-term quantum computers.

<div class="alert alert-block alert-info">
    
⚠️ **Note:** The concept of optimization enters in two distinct ways in this notebook:

1. **Solving the Classical Optimization Problem:** Our primary goal is to solve a classical optimization problem by encoding it into quantum circuits and running an algorithm to find a solution. 

2. **Optimizing Circuit Performance:** There are multiple ways to implement a quantum circuit, each yielding the same result but varying in efficiency on real hardware. Here, we focus on optimizing the circuit implementation to improve its performance characteristics on quantum devices

To illustrate these concepts, we use the Quantum Approximate Optimization Algorithm (QAOA) as a practical example for exploring the general problem of circuit optimization, which is essential for extracting useful results from a quantum algorithm.

</div> -->

<!-- ### Key Components of QAOA

QAOA works by encoding the problem into a quantum circuit that evolves through a series of unitary operations. The algorithm iteratively adjusts these operations to find the optimal solution. The two main components of QAOA are:

1. **Cost Function Hamiltonian (HC):** This is the problem-specific part of the circuit that encodes the objective function of the optimization problem. The goal of the QAOA is to find the parameters that minimize the expectation value of the cost function, driving the quantum system towards the optimal solution.

2. **Mixer Hamiltonian (HM):** This component of the circuit drives the exploration of the solution space. It typically uses single-qubit rotations that evolve the quantum state away from an initial, easily-prepared state.

In this notebook, we will focus on optimizing the **cost function circuit** of QAOA. The cost function circuit is crucial because it directly relates to the problem you want to solve. Specifically, the circuit is constructed to reflect the objective function of your optimization problem. -->


<!-- 
### Why Optimize the Cost Function Circuit?

Optimizing the cost function circuit is crucial because it directly impacts the performance and accuracy of QAOA. The circuit’s depth and the number of gates can significantly affect the algorithm's ability to find the optimal solution, especially on noisy quantum hardware. By applying optimization techniques during the transpilation process, we can reduce the circuit's complexity, making it more suitable for execution on real quantum devices.

In this notebook, we will use the [Qiskit SDK](https://docs.quantum.ibm.com/api/qiskit)'s transpiler to optimize the cost function circuit of QAOA, starting with a basic implementation, and gradually incorporating more advanced transpilation techniques and customizing our workflow.

Now, let's move on to implementing and optimizing the cost function circuit of QAOA using Qiskit SDK's transpiler! -->

<!-- ## 1. Problem Setting - Example: Max-Cut Problem

As a concrete example, we'll start our workflow with a graph that represents a hypothetical max-cut problem, where the goal is to divide the nodes in this graph into two sets such that the number of edges between the sets is maximized. The cost function Hamiltonian for our hypothetical Max-Cut problem is formulated as:

$$ H_C = \sum_{(i,j) \in E} \frac{1}{2} (1 - Z_i Z_j) $$

Here, $Z_i$ and $Z_j$ are Pauli-Z operators acting on qubits $i$ and $j$, and $E$ represents the set of edges in the graph. The cost function circuit for the Max-Cut problem thus involves a series of entangling gates (e.g., $RZZ$ gates defined as: $\text{RZZ}(\gamma) = e^{-i \gamma Z_i Z_j / 2}$  where $\gamma$ is a parameter to be optimized) that correspond to the edges in the graph. For references and more detail on the problem setting, you can find the tutorial on QAOA on the [learning plaform **here**](ttps://learning.quantum.ibm.com/tutorial/quantum-approximate-optimization-algorithm)  -->
### Operator (Hamiltonian)
$$ H_C = \sum_{(i,j) \in E} \frac{1}{2} (1 - Z_i Z_j) $$

The graph above corresponds to the following operators:

In [ ]:
from utils import build_max_cut_paulis
from qiskit.quantum_info import SparsePauliOp

local_correlators = build_max_cut_paulis(graph)
cost_operator     = SparsePauliOp.from_list(local_correlators)
num_qubits        = cost_operator.num_qubits
print('Number of qubits in operator (Hamiltonian): ', num_qubits)
print('\nDeglosed Operator (Each term of the Sum)\n')
print(f'Number of terms in operator {len(cost_operator)}\n')
print('Full operator:')
for n, oo in enumerate(cost_operator):
    print(f'        Operator {n}',oo)

### Ansatz (circuit)

Let’s construct the cost function circuit for this example. This operator maps to a 50-qubit QAOA circuit:

Now let's build the QAOA circuit/ansatz. We will use the [`QAOAAnsatz`](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.library.QAOAAnsatz) function in Qiskit to define it for us.

In [ ]:
from qiskit import QuantumCircuit
from qiskit.circuit.library import QAOAAnsatz
from qiskit.circuit import ParameterVector

# We are taking advantage of the QAOAAnsatz class to build the cost layer, 
# note that we are giving it dummy initial state and mixer circuits.

dummy_initial_state  = QuantumCircuit(num_qubits)  # the real initial state is defined later
dummy_mixer_operator = QuantumCircuit(num_qubits)  # the real mixer is defined later

# Use off-the-shelf qiskit QAOAAnsatz
ansatz = QAOAAnsatz(
    cost_operator,
    reps          = 1,
    initial_state = dummy_initial_state,
    mixer_operator= dummy_mixer_operator,
    name          = "QAOA cost block",
    )

ansatz.decompose().decompose().draw( fold=-1, scale=0.75,idle_wires=False)

## 2. Basic Transpilation with Preset Pass Manager

### Why Transpilation?

In quantum computing, a quantum circuit is a sequence of quantum gates applied to qubits to perform a computation. However, the circuit you initially design isn't always optimal for execution on a real quantum computer. This is where **transpilation** comes in.

### Transpilation Stages

1. **`init`:** This stage involves preparing the circuit by converting any custom instructions into single- and two-qubit gates, which are standard types of gates that the QPU can execute. This step also validates the circuit's instructions.

2. **`layout`:** During this stage, the virtual qubits in your circuit are mapped to the physical qubits on the QPU. This mapping is crucial because it determines how the qubits interact with each other based on the device's connectivity.

3. **`routing`:** After applying the layout, the circuit may need additional gates (such as SWAP gates) to ensure that qubits can interact according to the device's coupling map. This stage injects these gates to maintain the circuit's logical structure while adhering to the hardware's connectivity.

4. **`translation`:** In this stage, the circuit's gates are translated into the specific set of instructions (basis gates) that the QPU can execute. This ensures that the circuit is compatible with the device's Instruction Set Architecture (ISA).

5. **`optimization`:** This stage involves applying various optimization passes to reduce the circuit's depth and gate count. The goal is to find more efficient ways to represent the circuit that minimize the potential for errors and noise during execution.

6. **`scheduling`:** The final stage is scheduling, which organizes the execution of the circuit's instructions to account for idle times and other hardware constraints. This stage is particularly important for optimizing circuits on near-term devices.

For more details on the specifics of each of these stages, **check out this documentation link on [IBM Quantum Documentation](https://docs.quantum.ibm.com/guides/transpiler-stages).**

Let’s now select a target IBM Quantum backend from the Qiskit Runtime service to optimize our circuits. The backend target we are selecting is `ibm_brisbane`.


### Using Preset Pass Managers

Qiskit provides a convenient way to perform transpilation through **preset pass managers** using [`generate_preset_pass_manager`](https://docs.quantum.ibm.com/guides/defaults-and-configuration-options). These are predefined sets of transpilation passes that optimize the circuit to varying degrees depending on the selected **optimization level**:

- **Level 0:** *[No Optimization]* - Basic translation with trivial qubit mapping and no optimizations, primarily for hardware characterization.
- **Level 1:** *[Light Optimization]* - Reduces gate count and simplifies the circuit with minimal compile time, using basic layout and gate optimization.
- **Level 2:** *[Medium Optimization]* - Applies more sophisticated heuristic-based optimizations to further reduce circuit complexity. You can find more specifics in the documentation link [here](https://docs.quantum.ibm.com/guides/set-optimization)
- **Level 3:** *[High Optimization]* - Performs the most extensive optimizations, including advanced gate resynthesis and in-depth layout adjustments.

Each optimization level offers a different balance between the quality of the optimized circuit and the time it takes to transpile. Higher levels generally produce better optimized circuits but require more time to complete.

<div class="alert alert-block alert-info">

💡 **Why Use `generate_preset_passmanager` Instead of `transpile`?**

You may wonder why we recommend `generate_preset_passmanager` over the traditional `transpile` function. <b>Here’s why:</b> While `transpile` offers general optimization, `generate_preset_passmanager` provides greater flexibility and customization specifically tailored to different quantum hardware. With it, you can more precisely control aspects like gate reduction, error suppression, and qubit routing, helping you build circuits optimized for specific backend or execution requirements.
 
We encourage you to explore `generate_preset_passmanager` for a more powerful, flexible and customizable approach to circuit optimization, adding targeted enhancements to your quantum development workflow.
</div>


In this section, you will apply a preset pass manager to the QAOA cost function circuit designed to solve the Max-Cut problem. Initially, the circuit will be in its raw form, containing standard quantum gates like Hadamard and CNOT. By applying different levels of optimization, you will observe how the circuit is transformed into a more hardware-efficient version that uses gates supported by the chosen backend, such as RZ, X, SX, and ECR gates.

In [ ]:
# Pre-transpilation circuit
print(ansatz.decompose(reps=2).count_ops())

### How generate_preset_passmanager works? 


In [ ]:
from qiskit import QuantumCircuit
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

qc=QuantumCircuit(13)

for i in range(qc.num_qubits-1):
    qc.h(i)
    qc.cx(i,i+1)

######################################################### How transpilation works
# STEP1: invoke the pass_manager
mypm = generate_preset_pass_manager(optimization_level=3, 
                                    backend=backend,
                                    )
# STEP2: run the transpilation and name it as your "new" circuit
transpiled_qc = mypm.run(qc)

######################################################### DONE
print('Before transpilation:')
print('         ',qc.decompose(reps=2).count_ops())
print('After transpilation:')
print('         ',transpiled_qc.count_ops())


### Exercise 1: Explore Transpilation Levels
<div class="alert alert-block alert-success">
    
<!-- <b> Exercise 1: Explore Transpilation Levels</b>  -->

Now that you understand the basics of transpilation and the role of preset pass managers, it’s time to put this knowledge into practice. <b>Try transpiling the QAOA circuit you created earlier using `generate_preset_pass_manager`, and explore different `optimization_levels`.</b> Observe how the circuit changes and consider which level of optimization provides the best balance between transpilation time and circuit efficiency for your specific problem.
</div> 

In [ ]:
# Import necessary libraries
import time
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager


##################################
### Write your code below here ###

# Generate a preset pass manager for transpilation


### Don't change any code past this line ###
##################################
###############  Measure the start time of the transpilation process
t0 = time.time()

# Run the pass manager to transpile the QAOA cost layer circuit
naively_transpiled_qaoa = pm_ex1.run(ansatz)
print('Before transpilation:')
print('         ',ansatz.decompose(reps=2).count_ops())
# Measure the end time of the transpilation process
t1 = time.time()
print('After transpilation:')
print('         ',naively_transpiled_qaoa.count_ops())
# Print the total transpilation time
print(f"          Transpilation time: {t1 - t0:.2f} (s)")

In [ ]:
# Extract information from the transpiled circuit
circuit = naively_transpiled_qaoa
print(circuit.num_qubits)
circuit_characteristics(circuit)

#### Transpiling the Operator (Hamiltonian)

In [ ]:
isa_op = cost_operator.apply_layout(layout=naively_transpiled_qaoa.layout)
isa_op.num_qubits

## 3. Customize Transpilation Settings in `generate_preset_pass_manager`

### Stage1 : init (Initialization stage)
---

* Optimization,
* Multi-qubit operations (gates) $\rightarrow$ 1- and 2-qubit operations. These kindof operations are the ones that the backend can perform.

This stage has only one option "default", so there is not much to do.
#### Virtual Circuit

In [ ]:
from qiskit import  QuantumCircuit
import numpy as np
########################### GATE
mygate = QuantumCircuit(3, name =r'my-gate' )
mygate.mcrx(theta      = np.pi,
            q_controls = [0,1],
            q_target   = 2)
mygate.to_gate()

########################### CIRCUIT
qc = QuantumCircuit(3)
# 2 consecutive Hadamard gates
qc.h(0)
qc.h(0)

qc.barrier()
# apply the CRX gate
qc.append(mygate, [0,1,2])
print('VIRTUAL CIRCUIT')
qc.draw('mpl',initial_state=True)

#### Generate the pass manager & Transpilation

In [ ]:
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

print('backend name:',backend.name,'\nnum of qubits:',backend.num_qubits)

pm = generate_preset_pass_manager(backend     = backend,
                                  init_method = 'default',
                                  seed_transpiler = 12345)


isa_qc = pm.init.run(qc)

print('PHYSICAL CIRCUIT')
isa_qc.draw(fold = -1,output= 'mpl')

### Stage 2: Layout (placement)
---

Maps the virtual qubits to the physical qubits (backend qubits).

1. Mimics the virtual circuit as in the physical, i.e., from n-virtual qubits to backend-qubits.
2. Expands the virtual circuits operations to the backend-native operations.

**Layout methods**
* **default**: When `optimization_level=0`, 1st performs a `VF2Layout` to find a "perfect", if not it performs a `SabreLayout`.

* **dense** (DenseLayout): Searchs for the greatest number of available connection (dense) in the backend qubits.

* **trivial** (TrivialLayout): The $q_{i}^{virtual} \rightarrow q_{i}^{physical}$. Is one-to-one mapping. Hint: you can add a layout in the `QuantumCircuit`.

* **sabre** (SabreLayout): To choose an initial layout, then improves the circuit by doing a `Routing` (explained in the next step), reverse the circuit and apply `Routing` again. It depends of the `optimization_level` to try random initial layouts. For `optimization_level=0` only runs sabre, but for `optimization_level != 0` its tries to find the "perfect" layout, similar to **default**.

**Brisbane layout**

ADD the FIGURE

#### Generate the pass manager & Transpilation

In [ ]:
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
print('backend name',backend.name,'\nnum of qubits',backend.num_qubits)
pm = generate_preset_pass_manager(backend            = backend,
                                  layout_method      = 'default',
                                  # initial_layout     = [117,118,110],
                                  seed_transpiler    = 12345)

isa_qc = pm.layout.run(qc)

print('Circuit depth:',isa_qc.depth())

isa_qc.draw('mpl',fold=-1,idle_wires=False)


#### Task

<div class="alert alert-block alert-success">

Transpile with different layout methods (5 minutes):

* Use seed = 12345
* Don't forget the backend
<div>

In [ ]:
layout_options = ['default','dense','trivial','sabre']
for ll in layout_options:
  print('\033[1m'+'Layout Method:',ll)
  ##################################
  ### Write your code below here ###

  
  ### Don't change any code past this line ###
  ##################################
  isa_qc = pm.layout.run(qc)
  display(isa_qc.draw('mpl',fold=-1,idle_wires=False))
  print('\n    Circuit depth:', isa_qc.depth())
  print('---'*50)

### Stage3: Routing (Mapping or Swap-mapping) 
--- 
* ($Graph_{virtual}\rightarrow Graph_{physical}$)

Routing in your circuit ensures hardware compatibility and optimizes execution by inserting necessary swap gates. It also reduces errors by minimizing noise and maximizing fidelity.

Why apply swap gates?
* because multi-qubit gates, like CNOT, need to be connected. 

**Routing options**

* **basic**: Searches for qubit swaps following the most short rute, such that the connection to be executable in the device.

* **sabre**: Uses sabre algorithm to swap the 2-qubit gate operations.
* **stochastic**: Consider operations layer-by-layer, using a stochastic algorithm to find swap networks that implement a suitable permutation to make the layer executable.
* **lookahead**: Explore the nodes in the graph and heuristic techniques to swap gates and make them executable in the physical-graph.

#### Virtual Circuit

In [ ]:
from qiskit import  QuantumCircuit
import numpy as np
########################### CIRCUIT
qc = QuantumCircuit(30)
qc.h(0)
qc.h(0)
qc.barrier()
qc.y(qubit=1)
qc.rx(0.5,2)
qc.barrier()

for i in range(qc.num_qubits-1):
    qc.cx(i, i+1)
print('VIRTUAL CIRCUIT')
qc.draw('mpl',initial_state=True, fold=-1)

#### Generate the pass manager & Transpilation

In [ ]:
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
import numpy as np
pm = generate_preset_pass_manager(backend            = backend,     
                                  layout_method      ='default', 
                                  routing_method     ='basic' ,
                                  seed_transpiler    = 12345)

pre_isa_qc = pm.layout.run(qc)
isa_qc     = pm.routing.run(pre_isa_qc)
# display(isa_qc.decompose(reps=1).draw('mpl',fold=-1, idle_wires=False))
print('\n    circuit depth:', isa_qc.depth())
print('---'*50)

#### Task & Questions

<div class="alert alert-block alert-success">
Explore with the different routing methods. Which one is more effective? Which one takes less time?
<div>

In [ ]:
import time

routing_options = ['basic','sabre','stochastic','lookahead']
times = []
for oo in routing_options:
    print('\033[1m'+'layout method:',oo)
    ##################################
    ### Write your code below here ###

    
    ### Don't change any code past this line ###
    ##################################
    start      = time.time()
    pre_isa_qc = pm.layout.run(qc)
    isa_qc     = pm.routing.run(pre_isa_qc)
    dt         = time.time()-start
    times.append(dt)

    # display(isa_qc.decompose(reps=3).draw('mpl',fold=-1,idle_wires=False))
    circuit_characteristics(isa_qc)
    print('transpilation time (sec.):', dt)
    print('---'*50)
import matplotlib.pyplot as plt
plt.bar(routing_options,times)
plt.ylabel('time (sec.)')

### Stage4: Translation

Translates the gates in the virtual circuit to the ones natives in the backend.

**Translation methods**

* **translator**: Symbolic translation of gates to the target basis using known equivalences.
* **synthesis**:  Collect each run of one- and two-qubit gates into a matrix representation, and resynthesize from there.
#### Virtual Circuit

In [ ]:
from qiskit import  QuantumCircuit
import numpy as np
########################### CIRCUIT
qc = QuantumCircuit(30)
qc.h(0)
qc.h(0)
qc.barrier()
qc.y(qubit=1)
qc.rx(0.5,2)
qc.barrier()

for i in range(qc.num_qubits-1):
    qc.cx(i, i+1)
print('VIRTUAL CIRCUIT')
qc.draw('mpl',initial_state=True, fold=-1)

#### Generate the pass manager & Transpilation

In [ ]:
pm = generate_preset_pass_manager(
    backend=backend, 
    layout_method='sabre',
    routing_method='sabre',
    translation_method='synthesis', 
    seed_transpiler=12345)

pre_isa_qc=pm.init.run(qc)   # Stage 1
pre_isa_qc=pm.layout.run(pre_isa_qc)   #Stage 2
# pre_isa_qc = pm.routing.run(pre_isa_qc)  #Stage 3 
# pre_isa_qc = pm.translation.run(pre_isa_qc) #Stage 4
# pre_isa_qc = pm.optimization.run(pre_isa_qc) #Stage 5
isa_qc     = pm.translation.run(pre_isa_qc)

display(isa_qc.draw('mpl',fold=-1, ))
circuit_characteristics(isa_qc)
print('---'*50)

#### Task & Question
<div class="alert alert-block alert-success">

**7 minutes**

* Create a new circuit but instead of a Y-gate use a RZ-gate (or X-gate).

* What happends to the virtual gate?

* Explore the translation methods: 
    * translation methods ```['synthesis', 'translator'] ```
<div>

### Stage5: Optimization

The optimization stage in quantum circuit transpilation focuses on making low-level improvements, considering the specific characteristics of the hardware. This stage optimizes the circuits to work more efficiently on the quantum device.

Key points:

* Low-level optimization: Detailed adjustments are made to the circuit to enhance its performance on the specific hardware.
* ISA-compatible circuits: The input and output of this stage are circuits that are already compatible with the hardware's instruction set architecture (ISA).
* Iteration and adjustments: This stage often includes an optimization loop that can make multiple adjustments until the best possible result is achieved.

#### Virtual Circuit

In [ ]:
from qiskit import  QuantumCircuit

########################### CIRCUIT
qc = QuantumCircuit(3)
qc.h(0)
qc.h(0)
qc.barrier()
qc.y(qubit=1)
qc.barrier()

for i in range(2):
    qc.cx(i, i+1)
print('VIRTUAL CIRCUIT')
qc.draw('mpl',initial_state=True)

In [ ]:
pm = generate_preset_pass_manager(backend             = backend,      
                                #   optimization_method = 'default', 
                                  optimization_level  = 1,
                                  seed_transpiler     = 12345)
pre_isa_qc = pm.init.run(qc)   # Stage 1
pre_isa_qc = pm.layout.run(pre_isa_qc)   #Stage 2
pre_isa_qc = pm.translation.run(pre_isa_qc)
display(isa_qc.draw('mpl',fold=-1,idle_wires=False))
isa_qc     = pm.optimization.run(pre_isa_qc)

display(pre_isa_qc.draw('mpl',fold=-1,idle_wires=False))
circuit_characteristics(isa_qc)
print('---'*50)

#### Task
<div class="alert alert-block alert-success">

Repeat the transpilation for different values of `optimization_level`.

<div>

### Stage6: Scheduling

The Scheduling stage in quantum circuit transpilation is responsible for inserting explicit Delay instructions to account for idle periods of qubits. This stage ensures that the timing of operations is aligned with the hardware's constraints.

Key points:

* Explicit Delays: Adds Delay instructions to make idle times explicit.
* Timing Constraints: Ensures operations meet the hardware's timing requirements.
* Dynamical Decoupling: May include techniques like dynamical decoupling to reduce errors.


In [ ]:
from qiskit import  QuantumCircuit
import numpy as np
########################### CIRCUIT
qc = QuantumCircuit(3)
# 2 consecutive Hadamard gates
qc.h(0)
qc.h(0)
qc.barrier()
qc.y(qubit=1)
qc.barrier()

for i in range(qc.num_qubits-1):

    qc.cx(i, i+1)
    if i+2 < qc.num_qubits:
        qc.cx(i+1, i+2)
print('VIRTUAL CIRCUIT')
qc.draw('mpl',initial_state=True)

In [ ]:
pm = generate_preset_pass_manager(backend             = backend,      
                                  optimization_level  = 0,
                                  scheduling_method   = 'asap',
                                  seed_transpiler     = 12345)
pre_isa_qc = pm.run(qc)

isa_qc = pm.scheduling.run(pre_isa_qc)

display(isa_qc.decompose(reps=3).draw('mpl',fold=-1, idle_wires=False))
circuit_characteristics(isa_qc)
print('---'*50)

---

###  Optional Homework 1 - (Using the QAOA-ansatz)

<div class="alert alert-block alert-success">

Next, consider customizing some of the parameters available in the `generate_preset_pass_manager` function to see how they affect the transpilation process. You can adjust settings related to the circuit's `init`, `layout`, `routing`, `translation`, `optimization`, and `scheduling` stages.

<!-- Explore changing parameters pertaining to the transpilation stages and observe how these changes impact circuit metrics:

- **Init Stage:** Customize the synthesis algorithms with hls_config, select a different initialization plugin using `init_method`, or adjust the unitary synthesis method with `unitary_synthesis_method` and `unitary_synthesis_plugin_config`.
- **Layout Stage:** Modify the `layout_method` to change how the initial layout of qubits is selected. Options include `trivial`, `dense`, `noise_adaptive`, or `sabre`.
- **Routing Stage:** Adjust the `routing_method` to influence how qubits are swapped during the circuit execution. Options include `basic`, `lookahead`, `stochastic`, `sabre`, or `none`.
- **Translation Stage:** Experiment with the translation_method to determine how the circuit is converted to the backend’s native gate set, choosing between `translator` or `synthesis`.
- **Optimization Stage:** Experiment with the `approximation_degree` to trade off accuracy for gate reduction. You can set it to a value between `0.0` (maximal approximation) and `1.0` (no approximation), or `None` to automatically match the error rate.
- **Scheduling Stage:** Control the timing of gate execution with the scheduling_method, selecting either `as_soon_as_possible` (ASAP) or `as_late_as_possible` (ALAP). -->

You can learn more about these options here in the [documentation](https://docs.quantum.ibm.com/guides/defaults-and-configuration-options).

This homework will save a file which you have to send it.

<div>

<!-- <div class="alert alert-block alert-success">
<b>Make the most of the in-person QDC experience</b>
    
Feel free to experiment with the preset PassManager to discover the different ways they can add value to your quantum projects. Please do not hesitate to reach out to the mentors and IBMers in the room to ask questions and learn how to unlock even more potential from each of these capablities!

</div> -->

**Debugging tip:** When customizing transpilation settings, it can be extremely helpful to print out the list of passes that will be applied. This allows you to verify that your custom passes are being included in the correct order and that no unwanted default passes are being added. You can do this by adding a callback function as follows:

In [40]:
def callback_func(pass_, dag, time, property_set, count):
    print(f"PASS {count}: {pass_.name()} in {time}")
    print(f"ops: {dag.count_ops()}")
    print(f"dep: {dag.depth()}")
    print(f"========")

Examining the pass list helps you understand exactly how your circuit is being transpiled and can be invaluable for troubleshooting unexpected results or optimizing the transpilation process. If you notice passes that shouldn't be there, or are missing passes that you expected to be there, you can adjust your `generate_preset_passmanager` configuration accordingly.

Experiment with modifying parameters associated with the transpilation stages to observe their impact on your circuit metrics:

In [ ]:
import time

from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

# Feel free to play around with these options
##################################
##################################
##################################
### Write your code below here ###
your_name='YOURNAME'
your_last_name='YOURLASTNAME'
instruction_durations=None
init_method=None
initial_layout=None
layout_method=None
routing_method=None
translation_method=None
scheduling_method=None
unitary_synthesis_method=None
unitary_synthesis_plugin_config=None
hls_config=None
optimization_method=None
seed=12345 #dont change the seed
pm_hw = generate_preset_pass_manager(
    optimization_level = 3,
    backend=backend,
    instruction_durations=instruction_durations,
    init_method=init_method,
    initial_layout=initial_layout,
    layout_method=layout_method,
    routing_method=routing_method,
    translation_method=translation_method,
    scheduling_method=scheduling_method,
    unitary_synthesis_method=unitary_synthesis_method,
    unitary_synthesis_plugin_config=unitary_synthesis_plugin_config,
    hls_config=hls_config,
    optimization_method=optimization_method
    approximation_degree=0,
    seed_transpiler=seed,
    )

### Don't change any code past this line ###
##################################
##################################
##################################
###############  Measure the start time of the transpilation process
t0 = time.time()

# Apply of the pass manager and name it as a circuit
isa_circuit = pm_hw.run(ansatz, callback=callback_func)

# Measure the end time of the transpilation process
t1 = time.time()

# Print transpilation time
print(f"Transpilation time: {t1 - t0} (s)")

circuit_characteristics(isa_circuit)
save_pm_settings(isa_circuit)

**Tip**

When working with quantum circuits, the process of transpilation involves many **stochastic passes, which means the results can vary each time you transpile the same circuit**. In practice, running the transpiler multiple times may lead to better results in terms of circuit depth, gate count, and overall optimization. This is because some of the randomization involved in the transpilation process might occasionally yield a more efficient circuit layout or routing. One effective method to achieve more predictable outcomes is to use the `seed_transpiler` option. By varying a seed, you can control the randomization of the transpilation process, making it easier to reproduce and compare results.

Perform the process multiple times and keep the best outcome based on your chosen metric (e.g., minimal gate count, execution time). This can be achieved by running a loop over several transpilation attempts, each with a fixed seed, and selecting the most optimized circuit.

### Task: Transpilation with Different Seeds

**Try transpiling the circuit multiple times to observe variations in the results, then choose the most optimal one.**

Here’s an example where we transpile a circuit multiple times, each time storing the result and comparing the transpiled circuits based on gate count:



<div class="alert alert-block alert-danger">
    
⚠️ **Disclaimer on using this strategy:** 

While transpiling a circuit multiple times can lead to better optimization outcomes, it's important to be mindful of scalability issues. Transpiling multiple times can significantly increase the total runtime, especially for large circuits. This method is more appropriate for smaller circuits or when higher levels of optimization are essential. Additionally, each transpilation pass involves computational overhead, and for larger circuits or when scaling to many circuits, performing multiple trials could potentially result in the workflow becoming inefficient and unscalable.
</div>

<a id="exercise2"></a>
<div class="alert alert-block alert-success">
    
<!-- <b> Exercise 3: Transpile Multiple times</b>  -->

It's time to explore the impact of running the transpiler multiple times. For this exercise, at least transpilation stages and transpile the same circuit multiple times, and aim to achieve an optimized version by selecting the best result from these  attempts. 
</div>

In [ ]:
import time
from colorama import Fore, Style  # Ensure colorama is imported for colored output
from IPython.display import clear_output
# Initialize the depth to a large number for comparison
depth = 2000

# Record the starting time of the transpilation process
t0 = time.time()

# Loop over a range of seeds to find the best transpilation result
depth_2q = 0
seeds    = range(0,50)

for seed in seeds:


    ##################################
    ### Write your code below here 



    
    
    ### Don't change any code past this line ###
    ##################################

    # Calculate the depth of the circuit, considering only two-qubit gates 
    tmp_depth_2q = isa_qc.depth(lambda x: x.operation.num_qubits==2)   ########### don't change this line
    # If the new circuit depth is less than the current best depth, update the depth and save the circuit
    if tmp_depth_2q < depth_2q or depth_2q==0:
        print('Best seed:',seed)
        print('2-quibit depth',tmp_depth_2q)

        circuit_routed = isa_qc
        depth_2q       = tmp_depth_2q
        seed_selected  = seed



    clear_output(wait=True)
###############  Measure the start time of the transpilation process
t1 = time.time()
print('-------------------Optimal circuit--------------------')
# Output the selected seed that resulted in the best transpilation result
print(f'Seed selected: {seed_selected}')

# Use the circuit with the best depth
circuit = circuit_routed.decompose(reps = 2)
# display(circuit)
# Print transpilation time
print(f"Transpilation time: {t1 - t0} (s)")

# Output various metrics about the transpiled circuit
print(f'Count Ops: {circuit.count_ops()}')
print(f'Total circuit depth: {circuit.depth()}')
print(f'Number of 2-qubit gates: {circuit.size(lambda x: x.operation.num_qubits == 2)}')

# Output the score and depth specifically for 2-qubit gates, in color
print(Fore.MAGENTA + Style.BRIGHT + f'2-qubit gate depth: {circuit.depth(lambda x: x.operation.num_qubits == 2)}')
print(Fore.GREEN + f'Score: {score_func(circuit)}')

---
---
---
---

## 4. Leveraging AI in Transpilation

For this section, we are providing you with beta access to the Qiskit Transpiler Service to experiment with the latest AI-powered transpiler capabilities in Qiskit. With this release, you can now leverage AI and IBM Cloud&reg; resources to optimize your quantum circuits more efficiently.

The Qiskit Transpiler Service allows you to choose the level of AI-powered capablities in your transpilation process through the `ai` parameter, which can be set to `"true"`, `"false"`, or `"auto"`:
- **"true"** enables AI-powered transpilation passes, such as AIRouting or AI-powered synthesis, based on the optimization level selected.
- **"false"** limits the transpiler to the latest standard Qiskit features without AI intervention.
- **"auto"** allows the service to decide between AI-powered or heuristic transpilation passes, depending on the complexity and structure of your circuit.

In this step, you'll transpile and optimize your circuit using the Qiskit Transpiler Service. We will be using the Qiskit Transpiler Service, which provides both heuristic and AI-powered transpilation capabilities on the cloud. We can leverage running transpilation tasks to benefit from IBM Quantum™ cloud resources and advanced AI-powered transpiler passes. **For more information, refer to the Qiskit Transpiler Service [documentation.](https://docs.quantum.ibm.com/transpile/qiskit-transpiler-service)**


We encourage you to explore these options to maximize the efficiency and performance of your quantum circuits on the IBM Quantum™ cloud.




<div class="alert alert-block alert-danger">

Please note, **these features are in beta**. Have fun experimenting and pushing the boundaries of quantum development, and let us know if you have any feedback on how to make this tool better!

</div>

<a id="exercise4"></a>
<div class="alert alert-block alert-success">
    
<b> Exercise 4: Exploring AI-powered transpilation</b> 

In this exercise, you'll use the Qiskit Transpiler Service with AI-powered transpilation. First, ensure you have the latest version of the Qiskit Transpiler Service installed. Continue using the `ibm_brisbane` backend and create a new transpiler configuration with `optimization_level = 3` and the AI flag enabled (use `ai=True`). Transpile your quantum circuit using this new configuration and compare the results (gate count, depth) with your previous transpilation. 

**Homework**, experiment embedding AI-powered passes like `AIRouting`, `AICliffordSynthesis`, `AILinearFunctionSynthesis`, and `AIPermutationSynthesis` using `PassManager`s. Documentation for the same can be found [here.](https://docs.quantum.ibm.com/guides/ai-transpiler-passes)
</div> 


In [ ]:
# !pip install qiskit_ibm_transpiler
# Setting up the Transpiler service
from qiskit_ibm_transpiler import TranspilerService
import time
##################################
### Write your code below here ###



### Don't change any code past this line ###
##################################

###############  Measure the start time of the transpilation process
t0_ai_transpile = time.time()

# Run the pass manager to transpile the QAOA cost layer circuit
circuit_ai_true = ts_ai.run(ansatz)

# Measure the end time of the transpilation process
t1_ai_transpile  = time.time()
time_with_ai     = t1_ai_transpile - t0_ai_transpile

You can also use AI-powered transpiler passes as a drop-in custom pass in a staged pass manager. Read the [documentation](https://docs.quantum.ibm.com/guides/ai-transpiler-passes) for more details.

In [ ]:
# Extract information from the transpiled circuit
circuit = circuit_ai_true

circuit_characteristics(circuit);

Let's do it with a ""descent""  pass manager



In [ ]:
pm = generate_preset_pass_manager(backend            = backend,
                                  layout_method      = 'sabre',
                                  routing_method     = 'sabre',
                                  translation_method = 'translator',
                                  optimization_level =  1,
                                  scheduling_method  = 'asap',
                                  seed_transpiler    = 1000
                                  )
isa_qc = pm.run(ansatz)


# Print transpilation time
circuit_characteristics(isa_qc);

## 5. Diving Deeper into Transpilation Using Custom Pass Managers

### Introduction to Transpilation Strategies: Cheat Sheet

In quantum computing, optimizing circuits is crucial for improving the performance and accuracy of quantum algorithms on real hardware. While preset pass managers in Qiskit provide a solid foundation for general optimization, there are cases where more targeted, deeper strategies are needed to achieve better results. By customizing transpilation through specific strategies, you can refine circuit performance by reducing gate count, minimizing circuit depth, and optimizing qubit mapping for your specific hardware.

Let us dive deeper into these strategies and leverage Qiskit SDK's `PassManager`, to experiment with various techniques to improve the efficiency of your circuits, enabling them to run more effectively on QPUs.

### General Strategies Cheat Sheet

| **Strategy**                             | **When to Use This Strategy?**                                                        | **Why?**                                                                                                                                        | **How Much Improvement?**                                                                                       | **Implementation**                                                                                                                        |
|------------------------------------------|--------------------------------------------------------------------------------------|------------------------------------------------------------------------------------------------------------------------------------------------|-----------------------------------------------------------------------------------------------------------------|-----------------------------------------------------------------------------------------------------------------------------------------|
| **Try transpiling multiple times**       | Always (May not be needed in most cases now. Use `SabreLayout` and increase `trial_count` argument instead.) | Many transpilation passes are stochastic. Running the transpiler multiple times may result in better outcomes.                                                     | Transpile multiple times in a loop and keep the best result.                                                   | Utilize a loop or a genetic algorithm to mutate and evolve a better transpiler workflow.                                                                       |
| **Experiment with different synthesis methods** | When working with high-level objects that have multiple synthesis options (e.g., Cliffords, LinearFunctions, MCX gates, sequences of Pauli rotations) | Different synthesis methods can produce circuits with varying depths and gate counts without changing the operator. | Can optimize circuit depth and size without modifying functionality. | Specify different synthesis methods using `hls_config` in Qiskit. |
| **Dropping the final Clifford (operator backpropagation)** | When measuring an observable                                                  | Can potentially reduce the circuit depth with minimal overhead cost. | May help optimize depth without much extra complexity. | Implement operator backpropagation by rearranging gate sequences.                                                                                                                                         |
| **Use approximation degree**               | For deep circuits                                                                    | Deep circuits often accumulate noise, so trading off accuracy with fewer gates/errors may improve results.         | Can reduce the depth by optimizing gates based on the error rates of qubit gates. | Use `approximation_degree='none'` or set it based on hardware error rates.                                                                                                                                         |
| **Apply dynamical decoupling**              | For devices with high crosstalk errors (e.g., Eagle); may not be effective in all devices            | Dynamical decoupling can suppress crosstalk errors between qubits.                                                                                               | Potential improvements for specific hardware architectures. | Use Qiskit Runtime’s dynamical decoupling capabilities or create a custom decoupling pass.                                                                                                                              |
| **Numerical approximate methods**        | Large networks of CNOTs or Clifford gates                                             | Useful when dealing with many gates on relatively few qubits.                                                                                           | Can simplify circuits with large gate counts. | Explore numerical approximation methods for gate simplification.                                                                                                                                         |
| **Use SabrePreLayout**                   |  When the quality of the circuit (both size and depth) is more important than the transpilation runtime        | Helps reduce overall circuit depth by optimizing layout before routing.                                                                                               | Useful when depth is a more critical factor than size. | Useful for niche set of applications                                       |
| **Leverage Pauli synthesis tools**       | For circuits with large sequences of multi-qubit Pauli rotations or Trotter decomposition problems. | Optimizing Pauli rotations can significantly reduce circuit depth, especially in hardware with limited connectivity. | May result in substantial depth reduction in circuits with many Pauli rotations. | Use Pauli synthesis tools in Qiskit or Rustiq for Trotter circuits.                                                                                             |
| **Use approximation degree** |	For deep circuits where a trade-off between accuracy and circuit depth is acceptable (Approximates the operator)	| Trading off accuracy with fewer gates/lower depth may improve results. |	Can significantly reduce depth by allowing approximate gate synthesis.	| Use the approximation_degree parameter in Qiskit's transpiler. |



---




### Applying Transpilation Strategies Using Custom and Staged Pass Managers

In this section, you’ll learn how to integrate these optimization strategies into your custom pass manager to improve your quantum circuit. In Qiskit SDK, Custom `PassManagers` allow fine-grained control over the transpilation process by enabling users to define a specific sequence of optimization and transformation passes. These passes can target specific goals such as reducing circuit depth, improving fidelity, or adapting to the hardware topology.  By applying a combination of these methods, you can see how each strategy contributes to the overall circuit efficiency, making it more suitable for execution on IBM Quantum backends.

The `StagedPassManager` organizes the transpilation workflow into distinct stages, each stage being a `PassManager` focusing on different optimization goals, such as initial layout selection, gate reduction, or final qubit mapping. This staged approach allows for incremental improvements and conditional passes that apply only when necessary, offering a structured way to fine-tune the balance between execution speed and circuit fidelity. Together, the custom `PassManager` and `StagedPassManager` enable deeper exploration of transpilation strategies, giving developers full control over the optimization process to create circuits best suited for specific hardware constraints.

### Custom Pass Managers

As mentioned above, a **`PassManager`** allows you to apply a sequence of optimization passes to a quantum circuit, transforming it for more efficient execution on quantum hardware. By creating a custom pass manager, you can tailor the optimization to your circuit's specific needs, achieving deeper optimization compared to the preset configurations. Let's look at how we can create a custom `PassManager` and apply it to a workflow.

#### Example: Creating a Custom Pass Manager

To create a custom pass manager, you can combine various strategies together to form a transformation. Here is an example using `Collect2qBlocks`, `ConsolidateBlocks`, and `UnitarySynthesis` to optimize the circuit. You can refer to the [documentation](https://docs.quantum.ibm.com/api/qiskit/transpiler_passes) to know more about their transformation.


In [ ]:
backend.operation_names

In [55]:
from qiskit.transpiler import PassManager
# Optimization pass managers
from qiskit.transpiler.passes import Collect2qBlocks, ConsolidateBlocks
# Routing pass managers
from qiskit.transpiler.passes import BasicSwap, LookaheadSwap, StochasticSwap,SabreLayout, Commuting2qGateRouter,StarPreRouting
# Translation pass managers
from qiskit.transpiler.passes import  UnitarySynthesis, BasisTranslator

# Show example 
basis_gates = ["x", "sx", "ecr","rz"]

custom_pass_manager = PassManager([Collect2qBlocks(), #optimization
                                   ConsolidateBlocks(basis_gates=basis_gates), # optimization
                                   UnitarySynthesis(basis_gates) #init stage
                                   ])


You can then apply this custom pass manager to a circuit using its run method.

In [ ]:

###############  Measure the start time of the transpilation process
t0 = time.time()

# Run the pass manager to transpile the QAOA cost layer circuit
transpiled_circuit = custom_pass_manager.run(ansatz)
transpiled_circuit = transpiled_circuit.decompose(reps=4)

# Measure the end time of the transpilation process
t1 = time.time()

# Print transpilation time
print(f"Transpilation time: {t1-t0} (s)")

display(transpiled_circuit.draw(output="mpl",idle_wires=False, fold=-1,scale=0.5))
circuit_characteristics(transpiled_circuit);

In [ ]:
custom_pass_manager2 = PassManager([StochasticSwap(coupling_map=backend.coupling_map, seed=12345)])
isa_pm2  = custom_pass_manager2.run(transpiled_circuit)
display(isa_pm2.draw(idle_wires=False, fold=-1,scale=0.5))
circuit_characteristics(isa_pm2);

In [ ]:
from qiskit.circuit.equivalence_library import SessionEquivalenceLibrary

custom_pass_manager3 = PassManager(BasisTranslator(target_basis = basis_gates, 
                                                   equivalence_library = SessionEquivalenceLibrary))
isa_pm3  = custom_pass_manager3.run(isa_pm2)

display(isa_pm3.draw(idle_wires=False, fold=-1,scale=0.15))

circuit_characteristics(isa_pm3);

### Staged Pass Managers
With a **staged pass manager**, you can organize multiple `PassManagers` into distinct stages of transpilation, such as **initialization, layout, routing, translation, optimization, and scheduling**. This provides more flexibility in how the transpilation process is structured.

You can create a staged pass manager by defining stages and assigning pass managers to each stage:

In [72]:
from qiskit.transpiler import PassManager, StagedPassManager
from qiskit.transpiler.passes import UnitarySynthesis, Unroll3qOrMore
# Preset pass manager is the staged pass manager and flow from there. Use the preset passmanager and swap a stage
init_pm      = PassManager([UnitarySynthesis(basis_gates=basis_gates, min_qubits=3), 
                            Unroll3qOrMore()])

translate_pm = PassManager([Collect2qBlocks(), 
                            ConsolidateBlocks(basis_gates=basis_gates)])

staged_pass_manager = StagedPassManager(stages=["init", "translation"], init=init_pm, translation=translate_pm)

In [ ]:
###############  Measure the start time of the transpilation process
t0 = time.time()

# Run the pass manager to transpile the QAOA cost layer circuit
transpiled_circuit = staged_pass_manager.run(ansatz)

# Measure the end time of the transpilation process
t1 = time.time()

# Print the total transpilation time
print(f"transpilation time: {t1 - t0:.2f} (s)")
transpiled_circuit.decompose().draw(output="mpl",idle_wires=False, fold=-1,scale=0.5)
circuit_characteristics(transpiled_circuit)

**Now that you have learned how to create a custom `PassManager`, `StagedPassManager`, and AI-assisted passes, let us put it to test on our cost function circuit.**



### Optional Homework 2 
<div class="alert alert-block alert-success">

<a id="exercise5"></a>

<b> Exercise 5: Exploring custom pass managers</b> 

In the next exercise, use all of the above techniques to get to a better score for optimizing the QAOA cost function circuit. This exercise will help you appreciate how the advanced optimization techniques provided by the Qiskit SDK's transpiler enhance the performance of quantum circuits. It will also provide a foundation for experimenting with more complex customizations in the Track B version of this notebook.

**Challenge:**
1. Use a preset `PassManager` or a custom `PassManager` to optimize the QAOA cost function transpilation as much as possible. You can experiment to design your own `StagedPassManager` and integrate your custom `PassManager` workflow.
2. Try to achieve the best possible performance by experimenting with different combinations of passes.
3. Feel free to make use of AI-powered transpiler passes or external plugins from the [Qiskit ecosystem](https://www.ibm.com/quantum/ecosystem?tag=Transpiler+plugin) to acheive the same.

**Hint:**  
Remember that sometimes adding too many passes can lead to diminishing returns or even degrade performance. Aim for a balance! Once you've finalized your pass manager, submit your optimized circuit for grading.

**Let’s apply these strategies to optimize your QAOA circuit!**
</div> 






#### Useful Resources for Transpiler Passes

- General documentation for passes: [Qiskit Transpiler Passes Documentation](https://docs.quantum.ibm.com/api/qiskit/transpiler_passes)

#### Specific Pass Categories:
- **Layout Passes**: [Layout Passes](https://github.com/Qiskit/qiskit/tree/main/qiskit/transpiler/passes/layout)
- **Routing Passes**: [Routing Passes](https://github.com/Qiskit/qiskit/tree/main/qiskit/transpiler/passes/routing)
- **Translation Passes**: [Translation Passes](https://github.com/Qiskit/qiskit/tree/main/qiskit/transpiler/passes/synthesis)
- **Optimization Passes**: [Optimization Passes](https://github.com/Qiskit/qiskit/tree/main/qiskit/transpiler/passes/optimization)


In [ ]:
# Import necessary libraries and transpiler passes
from qiskit.circuit import SessionEquivalenceLibrary
from qiskit.transpiler import PassManager, StagedPassManager
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.transpiler.preset_passmanagers.common import generate_unroll_3q, generate_embed_passmanager
from qiskit.transpiler.passes.layout import (
    CSPLayout, DenseLayout, SabreLayout, VF2Layout, TrivialLayout
)
from qiskit.transpiler.passes.routing import (
    BasicSwap, LookaheadSwap, SabreSwap, StochasticSwap, StarPreRouting
)
from qiskit.transpiler.passes.basis import BasisTranslator
from qiskit.transpiler.passes.optimization import (
    CommutativeCancellation, ConsolidateBlocks, Collect2qBlocks, Collect1qRuns, Optimize1qGates, Optimize1qGatesDecomposition, Optimize1qGatesSimpleCommutation
)
from qiskit.transpiler.passes.synthesis import UnitarySynthesis, HighLevelSynthesis


# Set up coupling map for transpilation stages
cm = backend.coupling_map

### Write your code below here ###

# Suggestive template, feel free to change!

# # Initialize a staged pass manager
pm_ex5 = generate_preset_pass_manager(backend=backend, optimization_level=1, seed_transpiler = 12345)

# # Step 1: Initialization - Replace n-qubit QAOA operations with their 2-qubit gate decompositions
# pm_ex5.init = None

# # Step 2: Layout - Initialize PassManager for layout and apply layout passs
# pm_ex5.layout = None

# # Step 3: Routing - Configure routing passes for qubit routing 
# pm_ex5.routing = None

# # Step 4: Translation - Translate gates into the target basis supported by the backend
# pm_ex5.translation = None

# # Step 5: Optimization - Add passes for optimizations
# pm_ex5.optimization =None

# # Step 6: Scheduling - Add passes for scheduling
# pm_ex5.scheduling = None


### Don't change any code past this line ###

###############  Measure the start time of the transpilation process
t0 = time.time()

# Run the pass manager to transpile the QAOA cost layer circuit
pm_aprox_transpiled_qaoa = pm_ex5.run(cost_layer, callback=callback_func)

# Measure the end time of the transpilation process
t1 = time.time()

# Print transpilation time
print(f"Transpilation time: {t1 - t0:.2f} seconds")

In [ ]:
circuit = pm_aprox_transpiled_qaoa

# Output various metrics about the transpiled circuit
print(f'Count Ops: {circuit.count_ops()}')
print(f'Total circuit depth: {circuit.depth()}')
print(f'Number of 2-qubit gates: {circuit.size(lambda x: x.operation.num_qubits == 2)}')

# Output the score and depth specifically for 2-qubit gates, in color
print(Fore.MAGENTA + Style.BRIGHT + f'2-qubit gate depth: {circuit.depth(lambda x: x.operation.num_qubits == 2)}')
print(Fore.GREEN + f'Score: {score_func(circuit)}')

# Additional information

**Created by:** Raúl Guerrero-Avilés, Junye Huang.


**Version:** 1.0.0